# MHRatioPy

Note:
Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint

# Import API key
from api_keys import g_key

In [23]:
# Part 1 data file
master_data_path = "Resources/master_data.csv"
lat_lng_path1 = "Resources/australian_postcodes_loc.csv"

# Read the metadata 
master_data_df = pd.read_csv(master_data_path)
lat_lng_df1 = pd.read_csv(lat_lng_path1)

In [24]:
master_data_df.shape

(2641, 21)

In [25]:
lat_lng_df1.shape

(18442, 8)

In [26]:
master_data_df.columns

Index(['Postcode', 'House', 'Semi-detached', 'Apartment', 'Other dwelling',
       'No bedrooms (studios, etc)', '1 bedroom', '2 bedrooms', '3 bedrooms',
       '4 bedrooms', '5 bedrooms', '6 or more bedrooms',
       'Couple with no children', 'Couple with children',
       'One parent with children', 'Other family', 'Sole person household',
       'Group household', 'People with Mental Health Conditions',
       'Total People', 'State'],
      dtype='object')

In [27]:
lat_lng_df1.columns

Index(['id', 'Postcodes', 'locality', 'state', 'lat', 'long', 'Lat_precise',
       'Long_precise'],
      dtype='object')

In [28]:
master_data_df.dtypes

Postcode                                 int64
House                                    int64
Semi-detached                            int64
Apartment                                int64
Other dwelling                           int64
No bedrooms (studios, etc)               int64
1 bedroom                                int64
2 bedrooms                               int64
3 bedrooms                               int64
4 bedrooms                               int64
5 bedrooms                               int64
6 or more bedrooms                       int64
Couple with no children                  int64
Couple with children                     int64
One parent with children                 int64
Other family                             int64
Sole person household                    int64
Group household                          int64
People with Mental Health Conditions     int64
Total People                             int64
State                                   object
dtype: object

In [29]:
lat_lng_df1.dtypes

id                int64
Postcodes         int64
locality         object
state            object
lat             float64
long            float64
Lat_precise     float64
Long_precise    float64
dtype: object

In [30]:
lat_lng_df1 = lat_lng_df1.drop_duplicates(subset=["Postcodes"], keep='first')

In [31]:
lat_lng_df1.shape

(3167, 8)

In [32]:
master_data_with_coords_df = pd.merge(master_data_df, lat_lng_df1, how="left", left_on=["Postcode"], right_on=["Postcodes"])
master_data_with_coords_df = master_data_with_coords_df.drop("Postcodes",axis=1)
master_data_with_coords_df.head()                                           

,Postcode,House,Semi-detached,Apartment,Other dwelling,"No bedrooms (studios, etc)",1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,...,People with Mental Health Conditions,Total People,State,id,locality,state,lat,long,Lat_precise,Long_precise
0,800,33,6,5876,4,172,1856,3749,2805,168,...,344,7149,7: Northern Territory,3.0,DARWIN,NT,-12.458684,130.836680,-12.393279,130.776661
1,810,14828,2642,5649,85,207,1768,8552,9360,5484,...,1971,34330,7: Northern Territory,7.0,ALAWA,NT,-12.381806,130.866242,-12.380000,130.873000
2,812,9601,1127,1549,134,67,448,3389,6590,2353,...,1076,18634,7: Northern Territory,24.0,ANULA,NT,-12.391249,130.890470,-12.392800,130.890000
3,820,4926,3337,6733,104,167,1463,7566,7580,2372,...,1112,19920,7: Northern Territory,38.0,BAGOT,NT,-12.428017,130.873315,-12.411487,130.855410
4,822,9415,655,168,348,216,873,3083,4392,1811,...,755,25304,7: Northern Territory,59.0,ACACIA HILLS,NT,-13.643283,136.947051,-12.754154,131.148940


In [33]:
master_data_with_coords_df.shape

(2641, 28)

In [34]:
master_data_with_coords_df = master_data_with_coords_df.dropna(subset=['lat', 'long'])

In [35]:
master_data_with_coords_df.shape

(2640, 28)

In [36]:
master_data_with_coords_df["Mental Health Ratio"] = master_data_with_coords_df["People with Mental Health Conditions"] / master_data_with_coords_df["Total People"]*100

In [37]:
master_data_with_coords_df

,Postcode,House,Semi-detached,Apartment,Other dwelling,"No bedrooms (studios, etc)",1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,...,Total People,State,id,locality,state,lat,long,Lat_precise,Long_precise,Mental Health Ratio
0,800,33,6,5876,4,172,1856,3749,2805,168,...,7149,7: Northern Territory,3.0,DARWIN,NT,-12.458684,130.836680,-12.393279,130.776661,4.811862
1,810,14828,2642,5649,85,207,1768,8552,9360,5484,...,34330,7: Northern Territory,7.0,ALAWA,NT,-12.381806,130.866242,-12.380000,130.873000,5.741334
2,812,9601,1127,1549,134,67,448,3389,6590,2353,...,18634,7: Northern Territory,24.0,ANULA,NT,-12.391249,130.890470,-12.392800,130.890000,5.774391
3,820,4926,3337,6733,104,167,1463,7566,7580,2372,...,19920,7: Northern Territory,38.0,BAGOT,NT,-12.428017,130.873315,-12.411487,130.855410,5.582329
4,822,9415,655,168,348,216,873,3083,4392,1811,...,25304,7: Northern Territory,59.0,ACACIA HILLS,NT,-13.643283,136.947051,-12.754154,131.148940,2.983718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636,7466,27,0,0,0,0,0,10,12,0,...,32,6: Tasmania,18397.0,GORMANSTON,TAS,-42.291313,145.807453,-42.079962,145.619833,21.875000
2637,7467,1572,25,31,14,5,84,319,987,218,...,1808,6: Tasmania,18398.0,LAKE MARGARET,TAS,-42.093728,145.529693,-41.993139,145.588060,12.942478
2638,7468,557,6,8,0,0,33,145,251,116,...,697,6: Tasmania,18400.0,MACQUARIE HEADS,TAS,-42.706031,145.537671,-42.215300,145.198122,5.308465
2639,7469,607,62,0,8,3,62,126,412,80,...,772,6: Tasmania,18402.0,GRANVILLE HARBOUR,TAS,-41.784391,145.201632,-41.804795,145.041446,15.803109


In [38]:
locations_all_postcode = master_data_with_coords_df[["lat", "long"]].astype(float)

In [39]:
locations_all_postcode

,lat,long
0,-12.458684,130.836680
1,-12.381806,130.866242
2,-12.391249,130.890470
3,-12.428017,130.873315
4,-13.643283,136.947051
...,...,...
2636,-42.291313,145.807453
2637,-42.093728,145.529693
2638,-42.706031,145.537671
2639,-41.784391,145.201632


In [40]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [41]:
#Figure(layout=FigureLayo# Customise the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig1 = gmaps.figure(layout=figure_layout)

In [42]:
# Convert Mental Health to float and store
# HINT: be sure to handle NaN values

mental_health_ratio_df = master_data_with_coords_df["Mental Health Ratio"].astype(float)


In [43]:

#Heatmap

#marker_layer = gmaps.symbol_layer(wa_pg10000_locations)
fig_mental_health = gmaps.figure(center = (-28, 132) ,zoom_level = 3.7)
mental_health_layer = gmaps.heatmap_layer(locations_all_postcode, weights=mental_health_ratio_df, 
                                 dissipating=False, max_intensity=500,
                                 point_radius = 1)
fig_mental_health.add_layer(mental_health_layer)
#fig_mental_health.add_layer(marker_layer)
fig_mental_health


C:\Users\dalem\anaconda3\envs\PythonData\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Figure(layout=FigureLayout(height='420px'))